In [ ]:
#----------------------------------------------------
# PHOTOMETRY V5p4 HST+JWST for sources detected in  3.3um image
#              Jimena Rodriguez
#                April-29-2024
#---------------------------------------------------
# Import Functions
#---------------------------------------------------
import os
import astropy
import photutils
import numpy as np
from astropy.io import fits, ascii
from astropy import wcs
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.table import QTable
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus, CircularAnnulus, CircularAperture
from photutils.aperture import ApertureStats
from photutils.aperture import aperture_photometry
from photutils.utils import calc_total_error
from astropy.stats import SigmaClip
from photutils.background import SExtractorBackground, Background2D, MedianBackground

from scipy.constants import c as speed_of_light
from acstools import acszpt
from astropy.time import Time

#----------------------------------------------------
#List of PHANGS-JWST cycle 1 galaxies
#----------------------------------------------------

galaxies=['NGC1087','NGC1300','NGC1365','NGC1385','NGC1433','NGC1512','NGC1566','NGC1672','NGC2835',
          'NGC3351','NGC3627','NGC4254','NGC4303','NGC4321','NGC4535','NGC5068','NGC0628','NGC7496','IC5332']


galaxy_names=['ngc1087','ngc1300','ngc1365','ngc1385','ngc1433','ngc1512','ngc1566','ngc1672','ngc2835',
             'ngc3351','ngc3627','ngc4254', 'ngc4303','ngc4321','ngc4535','ngc5068','ngc628','ngc7496','ic5332']


#----------------------------------------------------
# Create output folder
#----------------------------------------------------
output_folder="/astro/phangshst/jimena/Photometry/v5p4_src_335/"
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)


i=0

for galaxy in galaxy_names:
    
#----------------------------------------------------
# Open data folders
#----------------------------------------------------
    Galaxy_folder=galaxies[i]
    HST_folder="/astro/phangshst/HST/"+Galaxy_folder+"/"
    err_HST_folder="/astro/phangshst/HST/"+Galaxy_folder+"/wht/"
    JWST_folder="/astro/phangshst/v1p1/"+Galaxy_folder+"/"
    
#----------------------------------------------------
# Sets of bands observed in each galaxy
#---------------------------------------------------
    if galaxy=='ngc1433' or galaxy=='ngc1087'  or galaxy=='ngc7496'  or galaxy=='ngc4321' or  galaxy=='ngc4254' or galaxy=='ngc1365' or galaxy=='ngc1385' :
        bands=['f275w','f336w','f438w','f555w','f657n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
        
    if galaxy=='ngc1566' or galaxy=='ngc5068':
        bands=['f275w','f336w','f438w','f555w','f658n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
        
    if  galaxy=='ngc1512'  or galaxy=='ngc2835' or galaxy=='ngc3627'  or galaxy=='ngc4303'  or galaxy=='ic5332':
        bands=['f275w','f336w','f438w','f555w','f658n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
    
    if galaxy=='ngc4535':
        bands=['f275w','f336w','f438w','f555w','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
        
    if galaxy=='ngc628':
        bands=['f275w','f336w','f435w','f555w','f658n','f658n_c','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
        
    if galaxy=='ngc1300':
        bands=['f275w','f336w','f435w','f555w','f658n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
          
    if galaxy=='ngc3351' :
        bands=['f275w','f336w','f438w','f547m', 'f555w','f657n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']

    if galaxy=='ngc1672':
        bands=['f275w','f336w','f435w','f550m','f555w','f658n','f814w','f200w','f300m','f335m','f360m','f770w','f1000w','f1130w','f2100w']
    
#----------------------------------------------------
# F555W Apertures corrections
# These values were derived in
# https://ui.adsabs.harvard.edu/link_gateway/2022MNRAS.510...32D/doi:10.1093/mnras/stab3213
#----------------------------------------------------
    if galaxy=='ngc628':
        ac_555 = -0.75
    if galaxy=='ngc1365' or galaxy=='ngc1300':
        ac_555 = -0.61
    if galaxy=='ngc1385' or galaxy=='ngc4321':
        ac_555 = -0.69
    if galaxy=='ngc1512':
        ac_555 = -0.62
    if galaxy=='ngc1566' or galaxy=='ngc1433':
        ac_555 = -0.62
    if galaxy=='ngc2835' or galaxy=='ngc3637':
        ac_555 = -0.81
    if galaxy=='ngc4254':
        ac_555 = -0.8
    if galaxy=='ngc4303':
        ac_555 = -0.74
    if galaxy=='ngc4535':
        ac_555 = -0.71
    if galaxy=='ngc5068':
        ac_555 = -0.77
    if galaxy=='ngc7496':
        ac_555 = -0.54
    if galaxy=='ngc1087':
        ac_555 = -0.67
    if galaxy=='ngc3351':
        ac_555 = -0.68
    if galaxy=='ngc1672':
        ac_555 = -0.6
    if galaxy=='ic5332': 
        ac_555=-0.59
 

      
#----------------------------------------------------
# For NGC628 we have two catalogs so it's necesary to
# change the name needed for open the images
#----------------------------------------------------
        
    if galaxy=='ngc628':
        galaxy_name='ngc628c'
        galaxy_jwst='ngc0628'
    else:
        galaxy_name=galaxy
        galaxy_jwst=galaxy
    
#----------------------------------------------------
# Open source catalog and create the new photometry catalog
#----------------------------------------------------
    
    sources=ascii.read("/astro/phangshst/jimena/Photometry/final_src_f335m/v1p1/"+galaxy_jwst+"_sources_f335m.csv")
    positions_sk = SkyCoord(sources['raj2000'], sources['dej2000'], unit='deg')
    colnames=['raj2000','dej2000', 'ID_source']
    coldata=[sources['raj2000'],sources['dej2000'],sources['source_id']]
    cat=QTable(coldata,names=colnames)

#----------------------------------------------------
#File with vega zero points values:
#https://app.box.com/s/o634trsuvver01b7x7ft0lebnngdqoxj
#File with Foreground extintion values:
#https://app.box.com/s/fcwe2sktg3bf9skztuap3o9wob7v28hp
#----------------------------------------------------
    Vega_zp_file=ascii.read('/astro/phangshst/jimena/hst_header_files/header_info_'+galaxy_name+'_prime.txt')
    extintion_table=ascii.read('/astro/phangshst/jimena/hst_tables/phangs_hst_galactic_extinction_table.csv')
    extintion_table2=ascii.read('/astro/phangshst/jimena/hst_tables/foreground_exintion_new_filters.txt')
#----------------------------------------------------


    for band in bands:
        
#----------------------------------------------------
# Aperture correction for HST bands:
# according to the ReadMe file:
# https://app.box.com/s/6e3j2sx60afwshwiaijn5qrcnet6e08l
# The offsets we apply to the measured V-band aperture
#correction for NUV, U, B, and I bands are
# −0.19, −0.12, −0.03, and −0.12 mags, respectively.
# for F628n, F627n, F550m, F547m the values were
#derived using linear interpolation

#Aperture corrections for Nircam band:
#were derived as expained in R0driguez+25

# JWST Vega zero points from
# http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?mode=browse&gname=JWST&gname2=MIRI&asttype=

#----------------------------------------------------
            
        if band=='f275w':
            ac=ac_555-0.19
        if band=='f336w':
            ac=ac_555-0.12
        if band=='f435w':
            ac=ac_555-0.03
        if band=='f438w':
            ac=ac_555-0.03
        if band=='f547m':
            ac=ac_555-0.0061
        if band=='f550m':
            ac=ac_555-0.0098
        if band=='f555w':
            ac=ac_555
        if band=='f657n':
            ac= ac_555-0.055
        if band=='f658n' or 'f658n_c':
            ac= ac_555-0.054
        if band=='f814w':
            ac=ac_555-0.12
        if band=='f200w':
            ac=-0.63
            ZP_v=759.59/1e6
        if band=='f300m':
            ac=-0.68
            ZP_v=377.25/1e6
        if band=='f335m':
            ac=-0.66
            ZP_v=305.60/1e6
        if band=='f360m':
            ac=-0.67
            ZP_v=266.13/1e6
        if band=='f770w':
            ZP_v=64.83/1e6
        if band=='f1000w':
            ZP_v=38.99/1e6
        if band=='f1130w':
            ZP_v=30.42/1e6
        if band=='f2100w':
            ZP_v=9.11/1e6
            
#----------------------------------------------------
# Set the apertures and annulus used in each band
#----------------------------------------------------
        #--------------------------------------------
        # HST   
        #--------------------------------------------
        if band=='f275w' or band=='f336w' or band=='f435w' or band=='f438w' or band=='f547m' or band=='f550m' or band=='f555w' or band=='f658n' or band=='f658n_c' or band=='f657n' or band=='f814w':
            pix_as=0.03962  # pixel size in arcsec for HST
            r_aper_as=4*pix_as # aperture = 4 pixels =0.158"
            an_in=7.          
            an_out=9.
            an_ape_in=an_in*pix_as   #inner annulus radius
            an_ape_out=an_out*pix_as #outer annulus radius
            
        #--------------------------------------------
        # NIRCAM 
        #--------------------------------------------        
        if band=='f200w':
            pix_as=0.031 # pixel size in arcsec for Nircam Short wavelength 
            r_aper_as=4*pix_as # aperture = 4 pixels =0.124"
            an_in=2.
            an_out=3.
            an_ape_in=an_in*r_aper_as  #inner annulus radius
            an_ape_out=an_out*r_aper_as #outer annulus radius
            
        if band=='f300m' or band=='f335m' or band=='f360m':
            pix_as=0.063 # pixel size in arcsec for Nircam long wavelength 
            r_aper_as=4*0.031  #use the equivalent of 4 pixels in F200W; aperture =0.124"
            an_in=2.
            an_out=3.
            an_ape_in=an_in*r_aper_as #inner annulus radius
            an_ape_out=an_out*r_aper_as  #outer annulus radius
            
        #--------------------------------------------
        # MIRI
        #--------------------------------------------     

        if band=='f770w' or band=='f1000w' or band=='f1130w' or band=='f2100w':
            pix_as=0.11 # pixel size in arcsec for MIRI
            
            #The apertures in each MIRI band correspond to the 50% encircle energy value:
            # https://jwst-docs.stsci.edu/jwst-mid-infrared-instrument/miri-performance/miri-point-spread-functions
            # Latest updates: original 13 Nov 2018  ?
            if band=='f770w':
                r_aper_as=0.168  # aperture in arcsec for F770W
            if band=='f1000w':
                r_aper_as=0.209 # aperture in arcsec for F1000W
            if band=='f1130w':
                r_aper_as=0.236 # aperture in arcsec for F1130W
            if band=='f2100w':
                r_aper_as=0.420 # aperture in arcsec for F2100W

            an_in=2.
            an_out=3.
            an_ape_in=an_in*r_aper_as #inner annulus radius
            an_ape_out=an_out*r_aper_as #outer annulus radius
            
#----------------------------------------------------
# Open the Images
#----------------------------------------------------

        #--------------------------------------------
        # JWST NIRCAM 
        #--------------------------------------------  
            
        if band=='f200w' or band=='f300m' or band=='f335m' or band=='f360m':
            hdu = fits.open(JWST_folder+galaxy_jwst+'_nircam_lv3_'+band+'_i2d_align.fits')
            data = hdu['SCI'].data
            header = hdu['SCI'].header
            datan = hdu['ERR'].data
            
            to_MJy = header['PIXAR_SR'] # pixel area in units of steradians
            
            #XPOSURE gives the exposure time and 
            #PHOTMJSR gives the conversion from counts/s to megaJy/steradian
            # So 'counts' gives the conversion from megaJy/steradian, 
            # which are the JWST image units, to counts
            
            counts=header['XPOSURE']/header['PHOTMJSR'] 
            fg_ext=0.        # We consider that the foreground extintion in the IR is 0
            ZP_ab=3631./1e6  #AB Zero point
            
        #--------------------------------------------
        # JWST MIRI
        #--------------------------------------------  

        if band=='f770w' or band=='f1000w' or band=='f1130w' or band=='f2100w':
            hdu = fits.open(JWST_folder+galaxy_jwst+'_miri_lv3_'+band+'_i2d_align.fits')
            data = hdu['SCI'].data
            header = hdu['SCI'].header
            datan = hdu['ERR'].data
            
            to_MJy = header['PIXAR_SR'] # pixel area in units of steradians
            ac=0.       # For MIRI we consider apertures corresponding to the 50% encircle energy
                        # so we will considere ac=0, and at the end multiply by 2. 
            fg_ext=0.   # We consider that the foreground extintion in the IR is 0
            
            #XPOSURE gives the exposure time and 
            #PHOTMJSR gives the conversion from counts/s to megaJy/steradian
            # So 'counts' gives the conversion from megaJy/steradian, 
            # which are the JWST image units, to counts
                       
            counts=header['XPOSURE']/header['PHOTMJSR'] 
            ZP_ab=3631./1e6 #AB Zero point
            
        #--------------------------------------------
        # HST WFC3 images
        #--------------------------------------------  

        if (band=='f275w' or band=='f336w' or band=='f438w' or (band=='f555w' and galaxy != 'ngc1300') or
        (band=='f814w' and (galaxy != 'ngc628' and  galaxy != 'ngc1300' and galaxy!='ngc1672' ))):
            
            if galaxy=='ngc7496' or galaxy=='ngc1365':
                hdu = fits.open(HST_folder+'hlsp_phangs-hst_hst_wfc3-uvis_'+galaxy+'_'+band+'_v1_exp-drc-sci.fits')
                hdun = fits.open(err_HST_folder+'hlsp_phangs-hst_hst_wfc3-uvis_'+galaxy+'_'+band+'_v1_err-drc-wht.fits')
            elif galaxy=='ngc3627' or galaxy=='ngc628' :
                hdu = fits.open(HST_folder+'hlsp_phangs-hst_hst_wfc3-uvis_'+galaxy+'mosaic_'+band+'_v1_exp-drc-sci.fits')
                hdun = fits.open(err_HST_folder+'hlsp_phangs-hst_hst_wfc3-uvis_'+galaxy+'mosaic_'+band+'_v1_err-drc-wht.fits')
            else:
                hdu = fits.open(HST_folder+galaxy+'_uvis_'+band+'_exp_drc_sci.fits')
                hdun = fits.open(err_HST_folder+galaxy+'_uvis_'+band+'_err_drc_wht.fits')
                
            data = hdu[0].data
            header = hdu[0].header
            datan=hdun[0].data
            
            # the '_err-drc-wht.fits' are Inverse variance maps.
            idxstatserror = (np.isfinite(datan)) & (datan != 0.)
            datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])
            
            # PHOTFNU is the Inverse sensitivity, Jy*sec/e- 
            # the factor to_MJy gives the conversion form e-/s to megaJy
            to_MJy=header['PHOTFNU']/1e6
            
            # 'counts' gives the conversion from e-/s, 
            # which are the units of the HST image, to counts
            counts=header['EXPTIME']/header['CCDGAIN']
            
            # Zero points for Ab and vega system in magnitudes
            ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
            ZP_v_mag=Vega_zp_file['zpVEGA'][Vega_zp_file['filter']==band.upper()].value
            
            # Zero points for Ab and vega system in megaJy
            ZP_v=10**(ZP_v_mag[0]/2.5)*to_MJy
            ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy

            #foreground extintion:
            if band=='f275w': key='F275W_UVIS_CHIP2'
            if band=='f336w': key='F336W_UVIS_CHIP2'
            if band=='f438w': key='F438W_UVIS_CHIP2'
            if band=='f555w': key='F555W_UVIS_CHIP2'
            if band=='f814w': key='F814W_UVIS_CHIP2'
            fg_ext=extintion_table[key][extintion_table['gal_name']==galaxy_name]

        #--------------------------------------------
        # HST ACS images
        #-------------------------------------------- 


        if  galaxy == 'ngc628' or galaxy=='ngc1672' or galaxy=='ngc1300':
            if band=='f435w' or band=='f814w' or (band=='f555w' and  galaxy=='ngc1300'):
                
                if galaxy=='ngc628':
                    hdu = fits.open(HST_folder+'hlsp_phangs-hst_hst_acs-wfc_'+galaxy+'mosaic_'+band+'_v1_exp-drc-sci.fits')
                    hdun = fits.open(err_HST_folder+'hlsp_phangs-hst_hst_acs-wfc_'+galaxy+'mosaic_'+band+'_v1_err-drc-wht.fits')
                else:
                    hdu = fits.open(HST_folder+galaxy+'_acs_'+band+'_exp_drc_sci.fits')
                    hdun = fits.open(err_HST_folder+galaxy+'_acs_'+band+'_err_drc_wht.fits')
 
                data = hdu[0].data
                header = hdu[0].header
                datan=hdun[0].data
            
                # the '_err-drc-wht.fits' are Inverse variance maps.
                idxstatserror = (np.isfinite(datan)) & (datan != 0.)
                datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])
                
                # units of the HST images are e-/s, the factor to_MJy gives the conversion form e-/s to megaJy
                # PHOTFLAM is the Inverse sensitivity, ergs/cm2/A/e-
                # PHOTPLAM is the Pivot wavelength (Angstroms) 
                
                to_MJy=(header["PHOTFLAM"]*(header["PHOTPLAM"]**2))/(speed_of_light*1e10)* 1e17
                
                # 'counts' gives the conversion from e-/s, 
                # which are the units of the HST image, to counts  
                counts=header['EXPTIME']/header['CCDGAIN']
                
                # Zero points for Ab and vega system in magnitudes
                ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
                ZP_v_mag=Vega_zp_file['zpVEGA'][Vega_zp_file['filter']==band.upper()].value
                
                # Zero points for Ab and vega system in megaJy
                ZP_v=10**(ZP_v_mag[0]/2.5)*to_MJy
                ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy
                
                #foreground extintion:
                if band=='f435w': key='F435W_ACS'
                if band=='f555w': key='F555W_ACS'
                if band=='f814w': key='F814W_ACS'
                fg_ext=extintion_table[key][extintion_table['gal_name']==galaxy_name]
                
        #--------------------------------------------
        # HST WFC3 and ACS Halpha images
        #-------------------------------------------- 
        # F658N
        #--------------------------------------------

        if  band=='f658n' or band=='f658n_c':
            if galaxy=='ngc1300' or galaxy=='ngc1672':
                hdu = fits.open(HST_folder+galaxy+'_acs_'+band+'_exp_drc_sci.fits')
                
            #For NGC 628, the observations are from two fields and are not combined 
            #into a mosaic image as for the rest of the HST bands 
            elif galaxy=='ngc628' and band=='f658n_c':  
                hdu = fits.open(HST_folder+galaxy+'c_acs_f658n_exp_drc_sci.fits')
                
            # At the moment in which the photometry for Rodriguez+25 was done
            # only the south field of NGC2835 had been observed
            elif galaxy=='ngc2835':
                hdu = fits.open(HST_folder+galaxy+'s_uvis_f658n_exp_drc_sci.fits')
                
            else:
                hdu = fits.open(HST_folder+galaxy+'_uvis_'+band+'_exp_drc_sci.fits')
            data = hdu[0].data
            header = hdu[0].header
            
                            
            # 'counts' gives the conversion from e-/s to counts 
            counts=header['EXPTIME']/header['CCDGAIN']
            
            #--------------------------------------------
            # Error images in F658N are only available
            # for NGC1300 and NGC1672
            # for the rest of the galaxies the error images are estimated
            #--------------------------------------------
            if galaxy=='ngc1300' or galaxy=='ngc1672':
                hdun = fits.open(err_HST_folder+galaxy+'_acs_'+band+'_err_drc_wht.fits')
                datan=hdun[0].data
                idxstatserror = (np.isfinite(datan)) & (datan != 0.)
                datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])            
            else:
                sigma_clip = SigmaClip(sigma=5., maxiters=10)
                bkg_estimator = SExtractorBackground()
                bkg = Background2D(data, (30,30), filter_size=(3, 3),  fill_value=0.0,
                                      sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
                datan = calc_total_error(data, bkg.background, header['EXPTIME'])

            #the factor to_MJy gives the conversion form e-/s to megaJy
            to_MJy=(header["PHOTFLAM"]*(header["PHOTPLAM"]**2))/(speed_of_light*1e10)* 1e17
    
    
            #--------------------------------------------
            # Estimation of the Vega and AB systems zero points
            #--------------------------------------------
            date=Time(header["EXPSTART"],format='mjd', scale='utc')
            q= acszpt.Query(date=date.strftime('%Y-%m-%d'), detector="WFC", filt='F658N')
            zpt_table = q.fetch()
            ZP_v_mag=zpt_table['VEGAmag'].value
            ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
            ZP_v=10**(ZP_v_mag/2.5)*to_MJy
            ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy
            
            #--------------------------------------------
            # Foreground extintion
            fg_ext=extintion_table2['F658N'][extintion_table2['Galaxy']==galaxy.upper()]
            
        #-------------------------------------------- 
        # F657N
        #--------------------------------------------
            
        if  band=='f657n':
            if galaxy=='ngc1365':
                hdu = fits.open(HST_folder+galaxy+'n_uvis_'+band+'_exp_drc_sci.fits')
            else:
                hdu = fits.open(HST_folder+galaxy+'_uvis_'+band+'_exp_drc_sci.fits')
            data = hdu[0].data
            header = hdu[0].header
            
            # 'counts' gives the conversion from e-/s to counts 
            counts=header['EXPTIME']/header['CCDGAIN']
            
            #--------------------------------------------
            # Error images in F657N are only available for NGC3351
            # for the rest of the galaxies the error images are estimated
            #--------------------------------------------
            
            if galaxy=='ngc3351':
                hdun = fits.open(err_HST_folder+galaxy+'_uvis_'+band+'_err_drc_wht.fits')
                datan=hdun[0].data
                idxstatserror = (np.isfinite(datan)) & (datan != 0.)
                datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])
            else:
                sigma_clip = SigmaClip(sigma=5., maxiters=10)
                bkg_estimator = SExtractorBackground()
                bkg = Background2D(data, (30,30), filter_size=(3, 3),  fill_value=0.0,
                                      sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
                datan = calc_total_error(data, bkg.background, header['EXPTIME'])

            #--------------------------------------------
            # Vega and AB systems zero points
            #--------------------------------------------    
            to_MJy=(header["PHOTFLAM"]*(header["PHOTPLAM"]**2))/(speed_of_light*1e10)* 1e17
            ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
            ZP_v_mag=22.332
            ZP_v=10**(ZP_v_mag/2.5)*to_MJy
            ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy
            #--------------------------------------------
            # Foreground extintion
            fg_ext=extintion_table2['F657N'][extintion_table2['Galaxy']==Galaxy_folder]
            
        #--------------------------------------------
        # HST WFC3 F547M image
        #-------------------------------------------- 
                       
            
        if  band=='f547m':
            hdu = fits.open(HST_folder+galaxy+'_uvis_'+band+'_exp_drc_sci.fits')
            data = hdu[0].data
            header = hdu[0].header
            counts=header['EXPTIME']/header['CCDGAIN']
            hdun = fits.open(err_HST_folder+galaxy+'_uvis_'+band+'_err_drc_wht.fits')
            datan=hdun[0].data
            idxstatserror = (np.isfinite(datan)) & (datan != 0.)
            datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])
            to_MJy=(header["PHOTFLAM"]*(header["PHOTPLAM"]**2))/(speed_of_light*1e10)* 1e17
            ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
            ZP_v_mag=24.763
            ZP_v=10**(ZP_v_mag/2.5)*to_MJy
            ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy
            fg_ext=extintion_table2['F547M'][extintion_table2['Galaxy']==Galaxy_folder]
            
        #--------------------------------------------
        # HST WFC3 F550M image
        #-------------------------------------------- 
           
        if  band=='f550m':
            hdu = fits.open(HST_folder+galaxy+'_acs_'+band+'_exp_drc_sci.fits')
            data = hdu[0].data
            header = hdu[0].header
            counts=header['EXPTIME']/header['CCDGAIN']
            hdun = fits.open(err_HST_folder+galaxy+'_acs_'+band+'_err_drc_wht.fits')
            datan=hdun[0].data
            idxstatserror = (np.isfinite(datan)) & (datan != 0.)
            datan[idxstatserror] = 1./np.sqrt(datan[idxstatserror])
            to_MJy=(header["PHOTFLAM"]*(header["PHOTPLAM"]**2))/(speed_of_light*1e10)* 1e17
            date=Time(header["EXPSTART"],format='mjd', scale='utc')
            q= acszpt.Query(date=date.strftime('%Y-%m-%d'), detector="WFC", filt='F658N')
            zpt_table = q.fetch()
            ZP_v_mag=zpt_table['VEGAmag'].value
            ZP_ab_mag=-2.5*np.log10(header["PHOTFLAM"])-5*np.log10(header["PHOTPLAM"])-2.408
            ZP_v=10**(ZP_v_mag/2.5)*to_MJy
            ZP_ab=10**(ZP_ab_mag/2.5)*to_MJy
            fg_ext=extintion_table2['F550M'][extintion_table2['Galaxy']==Galaxy_folder]

#--------------------------------------------
# Photometry
#-------------------------------------------- 

        #-------------------------------------------------
        # mask nan and inf in the science and error images 
        # Transform everything to counts
        #------------------------------------------------ 

        mask=((np.isinf(data)) | (np.isnan(data)) | (np.isinf(datan)) | (np.isnan(datan)))
        data=data*counts
        datan=datan*counts


        w = WCS(header)
        x,y=skycoord_to_pixel(positions_sk, w)  # trasform sky coordinates to x, y positions in the image
        positions_xy=np.transpose((x, y))

        
        
        # Aperture and annulus in arcsec 
        apertures= SkyCircularAperture(positions_sk, r_aper_as*u.arcsec)
        annulus_aperture=SkyCircularAnnulus(positions_sk, r_in=an_ape_in*u.arcsec, r_out=an_ape_out*u.arcsec)
        
        # Annulus in pixels        
        annulus_aperture_xy=CircularAnnulus(positions_xy, an_ape_in/pix_as, an_ape_out/pix_as)
        # Create a mask for the annulus
        annulus_masks = annulus_aperture_xy.to_mask(method='exact')
        # Sigma clipping that will be applied to the annulus to remove possible sources  
        sigclip = SigmaClip(sigma=3.0, maxiters=5)

        # Photometry in the aperture
        phot=aperture_photometry(data, apertures, wcs=w, error=datan, mask=mask)
        # Aperture and Annulus Statistics
        aper_stats = ApertureStats(data, apertures, wcs=w, error=datan, sigma_clip=None, mask=mask)
        bkg_stats = ApertureStats(data, annulus_aperture, error=datan, wcs=w, sigma_clip=sigclip, mask=mask, sum_method='exact')
        
        
        bkg_median=bkg_stats.median
        bkg_median[np.isnan(bkg_median)]=0
       
        

        
        #------------------------------------------------ 
        #Estimate the 90th and 10th percentile of the background
        # that will be used later for the uncertainties estimation 
        #------------------------------------------------ 
        bkg_10=[]
        bkg_90=[]
        bkg_10_clip=[]
        bkg_90_clip=[]
        N_pixels_annulus=[]
        N_pixel_annulus_clipped=[]
        
        for mask_an in annulus_masks:
            annulus_data = mask_an.multiply(data)
            if annulus_data is not None:
                annulus_data_1d = mask_an.multiply(data)[(mask_an.multiply(data)!=0) & (np.isfinite(mask_an.multiply(data))) & (~np.isnan(mask_an.multiply(data)))]
                if len(annulus_data_1d)>0:
                    annulus_data_filtered=sigclip(annulus_data_1d, masked=False)
                    bkg_low, bkg_hi = np.quantile(annulus_data_1d, [0.1,0.9])
                    bkg_low_clip, bkg_hi_clip = np.quantile(annulus_data_filtered, [0.1,0.9])
                    bkg_10.append(bkg_low)
                    bkg_90.append(bkg_hi)
                    bkg_10_clip.append(bkg_low_clip)
                    bkg_90_clip.append(bkg_hi_clip)
                    N_pixels_annulus.append(len(annulus_data_1d))
                else:
                    bkg_low=0.
                    bkg_hi =0.
                    bkg_10.append(bkg_low)
                    bkg_90.append(bkg_hi)
                    bkg_10_clip.append(0.)
                    bkg_90_clip.append(0.)
                    N_pixels_annulus.append(0)
            else:
                bkg_low=0.
                bkg_hi =0.
                bkg_10.append(bkg_low)
                bkg_90.append(bkg_hi)
                bkg_10_clip.append(0)
                bkg_90_clip.append(0)
                N_pixels_annulus.append(0)
            
            
        #-------------------------------------------- 
        # estimate the area of the aperture and the annulus 
        area_aper=aper_stats.sum_aper_area.value
        area_aper[np.isnan(area_aper)]=0
        area_sky=bkg_stats.sum_aper_area.value
        area_sky[np.isnan(area_sky)]=0
        
        #-------------------------------------------- 
        # Estimate a total value for the background by multiplying the median of the background 
        # by the area of the aperture
        # Do the same process but using the 10 and 90 th percentiles
        # and also when sigma clipping is applied to the annulus before estimating the 10th and 90th percentiles 
        #-------------------------------------------- 
        
        total_bkg=bkg_median*area_aper
        total_bkg_10=bkg_10*area_aper
        total_bkg_90=bkg_90*area_aper
        total_bkg_10_clip=bkg_10_clip*area_aper
        total_bkg_90_clip=bkg_90_clip*area_aper
        
        bkg_std=bkg_stats.std
        bkg_std[np.isnan(bkg_std)]=0
        
        
        #--------------------------------------------  
        # Subtract the background value from the measurement in the aperture, 
        # divide by counts to recover megajy/sr units
        # Do the same process with the 10 and 90 th percentiles
        # with and without applying sigma clipping
        #-------------------------------------------- 
       

        flux_dens=(phot['aperture_sum'] - total_bkg)/counts       
              
        flux_dens_bkg_10=(phot['aperture_sum'] - total_bkg_10)/counts
        flux_dens_bkg_90=(phot['aperture_sum'] - total_bkg_90)/counts
        flux_dens_bkg_10_clip=(phot['aperture_sum'] - total_bkg_10_clip)/counts
        flux_dens_bkg_90_clip=(phot['aperture_sum'] - total_bkg_90_clip)/counts
        
        #-------------------------------------------- 
        # Error Estimation:
        # 'flux_dens_err' is the standard uncertainty, which accounts for 
        # Poisson noise and background error.
        # 'flux_dens_err_9010' and 'flux_dens_err_9010_clip' include an 
        # additional term that reflects the variation in measurement when 
        # using the 10th and 90th percentiles of the background. 
        # This difference corresponds to 4.6 sigma.
        # Therefore, we divide by 4.6 to obtain the 1 sigma uncertainty
        #-------------------------------------------- 
        
        flux_dens_err=np.sqrt(pow(phot['aperture_sum_err'],2.)+(pow(bkg_std*area_aper,2)/bkg_stats.sum_aper_area.value)*np.pi/2)/counts
        flux_dens_err_9010 = np.sqrt(flux_dens_err**2 + (flux_dens_bkg_10 - flux_dens_bkg_90)**2)/4.6
        flux_dens_err_9010_clip = np.sqrt(flux_dens_err**2 + (flux_dens_bkg_10_clip - flux_dens_bkg_90_clip)**2)/4.6
        
        
        #Sigma to Noise in each case
        s2n_old=flux_dens/flux_dens_err
        s2n=flux_dens/flux_dens_err_9010
        s2n_clip=flux_dens/flux_dens_err_9010_clip
        
        # put everything in megaJy and correct by aperture correction and foreground extintion 
        flux=flux_dens*to_MJy*10**((ac-fg_ext)/-2.5)
        flux_err=flux_dens_err*to_MJy
        flux_err_9010=flux_dens_err_9010*to_MJy
        flux_err_9010_clip=flux_dens_err_9010_clip*to_MJy

        # flux_bkg is the flux that I am sustraction from the aperture
        flux_bkg=(total_bkg/counts)*to_MJy*10**((ac-fg_ext)/-2.5)
        # flux_plus_bkg is the flux in the aperture before sustraction the background
        flux_plus_bkg=(phot['aperture_sum']/counts)*to_MJy*10**((ac-fg_ext)/-2.5)
        
        #---------------------------------------------------
        # JWST MIRI bands
        #For MIRI bands the photometry was performed with the aperture size corresponding 
        #to the 50% encircle energy. 
        #So we multiply by two to obtain the total energy of the source
        #--------------------------------------------------- 
        
        if band=='f770w' or band=='f1000w' or band=='f1130w' or band=='f2100w':
            flux=flux_dens*to_MJy*10**((ac-fg_ext)/-2.5)*2
            flux_err=2*flux_dens_err*to_MJy
            flux_err_9010=2*flux_dens_err_9010*to_MJy
            flux_err_9010_clip=2*flux_dens_err_9010_clip*to_MJy
            flux_bkg=2*(total_bkg/counts)*to_MJy*10**((ac-fg_ext)/-2.5)
            flux_plus_bkg=2*(phot['aperture_sum']/counts)*to_MJy*10**((ac-fg_ext)/-2.5)
            flux_err_delta_apertures=2*flux_err_delta_apertures



        #---------------------------------------------------
        # Transform to magnitude
        #---------------------------------------------------       
        mag_ab=-2.5*np.log10(flux/ZP_ab)
        mag_veg=-2.5*np.log10(flux/ZP_v)

        err_mag=abs((2.5/np.log(10))*(flux_err/flux))
        err_mag_9010=abs((2.5/np.log(10))*(flux_err_9010/flux))
        err_mag_9010_clip=abs((2.5/np.log(10))*(flux_err_9010_clip/flux))




#---------------------------------------------------
# Built catalog
# Multiply flux densities to 1E9 to convert to mjy
#---------------------------------------------------
        cat.add_column(flux*1E9,name='flux_'+band.upper())
        cat.add_column(flux_err*1E9,name='er_flux_'+band.upper())
        cat.add_column(flux_err_9010*1E9,name='er_flux_9010_'+band.upper())
        cat.add_column(flux_err_9010_clip*1E9,name='er_flux_9010_clip_'+band.upper())
        cat.add_column(s2n_old,name='signal_to_noise_original_'+band.upper())
        cat.add_column(s2n,name='signal_to_noise_9010_'+band.upper())
        cat.add_column(s2n_clip,name='signal_to_noise_9010_clip_'+band.upper())
        cat.add_column(flux_bkg*1E9, name='flux_bkg_'+band.upper())
        cat.add_column(flux_plus_bkg*1E9, name='flux_'+band.upper()+'_b')
        cat.add_column(mag_ab,name=band.upper()+'_ab')
        cat.add_column(mag_veg,name=band.upper()+'_veg')
        cat.add_column(err_mag,name='err_'+band.upper())
        cat.add_column(err_mag_9010,name='err_9010_'+band.upper())
        cat.add_column(err_mag_9010_clip,name='err_9010_clip_'+band.upper())

        

#---------------------------------------------------
# Save the catalog in ascii and fits format
#---------------------------------------------------
           
    cat.write(output_folder+'Phot_v5p4_src335_'+galaxy+'.csv',overwrite=True, format="ascii.csv")
    cat.write(output_folder+'Phot_v5p4_src335_'+galaxy+'.fits',overwrite=True)

  
    i+=1



In [2]:
from astropy.io import fits
galaxy='ngc7496'
band='f555w'
Galaxy_folder=galaxy.upper()
HST_folder="/astro/phangshst/HST/"+Galaxy_folder+"/"
#hdu = fits.open(HST_folder+'hlsp_phangs-hst_hst_wfc3-uvis_'+galaxy+'_'+band+'_v1_exp-drc-sci.fits')
header = hdu[0].header


In [3]:
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                10000                                                  
NAXIS2  =                10000                                                  
EXTEND  =                    T                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
FILENAME= 'ngc7496_uvis_f555w_exp_drc_sci.fits' / name of file                  
FILETYPE= 'SCI      '          / type of data found in data file                
                                                                                
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'WFC3  '          

In [7]:
galaxy='ngc1300'
Galaxy_folder=galaxy.upper()
HST_folder="/astro/phangshst/HST/"+Galaxy_folder+"/"
hdu = fits.open(HST_folder+galaxy+'_acs_'+band+'_exp_drc_sci.fits')
header = hdu[0].header
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                12000                                                  
NAXIS2  =                12000                                                  
EXTEND  =                    T                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
                                                                                
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'ACS   '             / identifier for instrument used to acquire data 
                                                                                
              / DATA DESCRIP